In [1]:
import time
from datetime import datetime, timezone

##  Allows a function to run only on odd seconds

In [15]:
def dec_odd_secs(fn):
    """
    Decorator to ensure that function that it decorates runs only on odd seconds.
    """
    import time
    from functools import wraps
    
    @wraps(fn)
    def inner(*args, **kwargs):
        secs = round(time.time(), 0)
        if secs % 2 == 0:
            result = f'Not right time to run {fn.__name__}. Odd times - {secs} !'
        else:
            result = fn(*args, **kwargs)
            print(f'{fn.__name__} ran in {secs}')
        return result
    return inner

In [16]:
@dec_odd_secs
def fn_odd_secs():
    """
    This is a function i.e. supposed to run only on odd seconds.
    """
    pass

In [24]:
fn_odd_secs.__doc__

'\n    This is a function i.e. supposed to run only on odd seconds.\n    '

In [57]:
if round(time.time(),0) % 2 != 0:
    print(f' Odd secs : {round(time.time(),0)}')
    fn_odd_secs()
else:
    print(f' Even secs : {round(time.time(),0)}')

 Odd secs : 1624526845.0
fn_odd_secs ran in 1624526845.0


In [2]:
def dec_take_time(secs):
    def dec_odd_secs(fn):
        """
        Decorator to ensure that function that it decorates runs only on odd seconds.
        """
        import time
        from functools import wraps

        @wraps(fn)
        def inner(*args, **kwargs):
            if secs % 2 == 0:
                result = f'Not right time to run {fn.__name__}. Odd times - {secs} !'
            else:
                result = fn(*args, **kwargs)
                print(f'{fn.__name__} ran in {secs}')
            return result
        return inner
    return dec_odd_secs

In [3]:
@dec_take_time(round(time.time(), 0))
def fn_odd_secs():
    """
    This is a function i.e. supposed to run only on odd seconds.
    """
    pass

In [18]:
fn_odd_secs()

fn_odd_secs ran in 1624976551.0


In [23]:
fn_odd_secs()

fn_odd_secs ran in 1624976561.0


## LOG

### This log will provide a time stamp while calling a function

In [49]:
## Logging for functions
def logger(fn:'function')->'function':
    """
    Decorator that keeps log of time at which function that it decorates is invoked."
    """
    from datetime import datetime, timezone
    from functools import wraps
    from time import perf_counter

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        time_called = datetime.now(timezone.utc)
        result = fn(*args, **kwargs)
        end = perf_counter()
        time_took = end - start
        str1 = f'Function name {fn.__name__} was called at {time_called} and result is **{result}**\n'
        str2 = f'Function description is {fn.__doc__}'
        str3 = f'Function annotation is {fn.__annotations__} and Execution time is {time_took}'
        msg = str1 + str2 + str3
        return msg
    return inner

In [50]:
@logger
def add(a:str, b:int)->str:
    """
    This is a function's writeup.
    This is a function that takes 2 variables var1 which is str and var2 which is int.
    Function is decorated to keep a log of time at which it is invoked.
    Function will return a string combining var1 and var2
    """
    result = f'{a} and {b} are input supplied'
    return result

In [51]:
add.__doc__

"\n    This is a function's writeup.\n    This is a function that takes 2 variables var1 which is str and var2 which is int.\n    Function is decorated to keep a log of time at which it is invoked.\n    Function will return a string combining var1 and var2\n    "

In [52]:
add('abc',10)

"Function name add was called at 2021-07-01 14:43:11.571378+00:00 and result is **abc and 10 are input supplied**\nFunction description is \n    This is a function's writeup.\n    This is a function that takes 2 variables var1 which is str and var2 which is int.\n    Function is decorated to keep a log of time at which it is invoked.\n    Function will return a string combining var1 and var2\n    Function annotation is {'a': <class 'str'>, 'b': <class 'int'>, 'return': <class 'str'>} and Execution time is 5.40000110049732e-06"

In [16]:
add.__annotations__

{'a': int, 'b': int, 'return': int}

### Below decorator will provide info of an object

In [25]:
def info(obj):
    logs = []
    logs.append(f' Object belongs to the class: {obj.__class__.__name__}')
    logs.append(f' Id of object : {hex(id(obj))}')
    logs.append(f' Object was called at : {datetime.now(timezone.utc)} UTC')
    return logs

In [26]:
def dec_debug(cls):
    cls.debug_info = info
    return cls

In [27]:
@dec_debug
class Country():
    def __init__(self, name, capital, population, GDP):
        self.name = name
        self.capital = capital
        self.population = population
        self.GDP = GDP
        
    def __call__(self):
        return f'Country is {self.name}'

In [28]:
India = Country('India', 'Delhi', '1.3B', '$3.05 t')

In [29]:
India()

'Country is India'

In [30]:
India.debug_info()

[' Object belongs to the class: Country',
 ' Id of object : 0x1e29e555ca0',
 ' Object was called at : 2021-06-29 15:22:45.859607+00:00 UTC']

In [31]:
len(India.debug_info())

3

## Authenticate - Will allow to run a function only once authenticated

In [131]:
def dec_factory(user_pwd):
    def dec_authenticate(fn):
        pwd = 'Buddha-Smiling'

        def inner(*args, **kwargs):
            if user_pwd == pwd:
                result = fn(*args, **kwargs)
                return f'Authentication success & result is {result}'
            else:
                return f'Authentication failed !!'   
        return inner
    return dec_authenticate

In [132]:
@dec_factory('anil')
def div(a, b):
    """
    This function is to divide 2 given numbers
    """
    return a/b

In [133]:
div(10, 2)

'Authentication failed !!'

In [137]:
@dec_factory('Buddha-Smiling')
def remainder(a, b):
    """
    This function is to divide 2 given numbers and give its remainder
    """
    return a%b

In [136]:
remainder(12, 2)

'Authentication success & result is 0'

### Timer - This decorator should run a function 'n' number of times and give back the average time took

In [27]:
##Version to understand the flow

In [28]:
def dec_factory(repeat):
    print('dec_factory')
    
    def dec_timed(fn):
        
        total_elapsed = 0
        total_cnt     = 0
        print('dec_timed')
        
        from functools import wraps
        from time import perf_counter
        @wraps(fn)
        def inner(*args, **kwargs):
            print('inner-in')
            nonlocal total_elapsed
            nonlocal total_cnt
            for i in range(repeat):
                start  = perf_counter()
                result = fn(*args, **kwargs)
                print(f'repeat:{i}, args:{args}, results:{result}')                
                end    = perf_counter()
                total_elapsed += (end-start)
                total_cnt     += 1
            print('inner-out')    
            avg_time = total_elapsed/total_cnt
            #return f'Function - {fn.__name__} ran {repeat} times & took {avg_time} secs. Result is {result}'
            return result
        print('dec_timed:out')
        return inner
    print('dec_factory:out')
    return dec_timed                

In [29]:
#with lru_cache

from functools import lru_cache

@dec_factory(4)
@lru_cache(maxsize=4)
def fact(n):
    print(f'Calculating fact({n})')
    return 1 if n < 2 else n * fact(n-1)

dec_factory
dec_factory:out
dec_timed
dec_timed:out


In [30]:
fact(3)

inner-in
Calculating fact(3)
inner-in
Calculating fact(2)
inner-in
Calculating fact(1)
repeat:0, args:(1,), results:1
repeat:1, args:(1,), results:1
repeat:2, args:(1,), results:1
repeat:3, args:(1,), results:1
inner-out
repeat:0, args:(2,), results:2
repeat:1, args:(2,), results:2
repeat:2, args:(2,), results:2
repeat:3, args:(2,), results:2
inner-out
repeat:0, args:(3,), results:6
repeat:1, args:(3,), results:6
repeat:2, args:(3,), results:6
repeat:3, args:(3,), results:6
inner-out


6

In [31]:
#without lru_cache. We can see more displays

@dec_factory(4)
def fact(n):
    print(f'Calculating fact({n})')
    return 1 if n < 2 else n * fact(n-1)

dec_factory
dec_factory:out
dec_timed
dec_timed:out


In [32]:
fact(3)

inner-in
Calculating fact(3)
inner-in
Calculating fact(2)
inner-in
Calculating fact(1)
repeat:0, args:(1,), results:1
Calculating fact(1)
repeat:1, args:(1,), results:1
Calculating fact(1)
repeat:2, args:(1,), results:1
Calculating fact(1)
repeat:3, args:(1,), results:1
inner-out
repeat:0, args:(2,), results:2
Calculating fact(2)
inner-in
Calculating fact(1)
repeat:0, args:(1,), results:1
Calculating fact(1)
repeat:1, args:(1,), results:1
Calculating fact(1)
repeat:2, args:(1,), results:1
Calculating fact(1)
repeat:3, args:(1,), results:1
inner-out
repeat:1, args:(2,), results:2
Calculating fact(2)
inner-in
Calculating fact(1)
repeat:0, args:(1,), results:1
Calculating fact(1)
repeat:1, args:(1,), results:1
Calculating fact(1)
repeat:2, args:(1,), results:1
Calculating fact(1)
repeat:3, args:(1,), results:1
inner-out
repeat:2, args:(2,), results:2
Calculating fact(2)
inner-in
Calculating fact(1)
repeat:0, args:(1,), results:1
Calculating fact(1)
repeat:1, args:(1,), results:1
Calculati

6

In [33]:
# Assignment version

def dec_factory(repeat):
    
    def dec_timed(fn):
        
        total_elapsed = 0
        total_cnt     = 0
        
        from functools import wraps
        from time import perf_counter
        @wraps(fn)
        def inner(*args, **kwargs):
            nonlocal total_elapsed
            nonlocal total_cnt
            for i in range(repeat):
                start  = perf_counter()
                result = fn(*args, **kwargs)           
                end    = perf_counter()
                total_elapsed += (end-start)
                total_cnt     += 1
            avg_time = total_elapsed/total_cnt
            return f'Function - {fn.__name__} ran {repeat} times & took {avg_time} secs. Result is {result}'
        return inner
    return dec_timed     

In [39]:
@dec_factory(100_000_000)
def add(a,b):
    return a + b

In [40]:
add(100, 200)

'Function - add ran 100000000 times & took 3.2433564499089697e-07 secs. Result is 300'

### Provides privilege access (has 4 parameters, based on privileges (high, mid, low, no), gives access to all 4, 3, 2 or 1 params)

In [53]:
user_list = {'Amit':'All', 
             'Amir':'Medical',
             'Anil':'Dental',
             'Sachin':'Vision'
                }

In [64]:
def dec_factory(privilege_lvl):

    def dec_privilege(fn):
        access = {'high':('add2', 'remove', 'modify', 'view'), 'mid':('add', 'modify', 'view'), 'low':('view'),'no':()}
        def inner(*args, **kwargs):
            available_fns = access[privilege_lvl]
            if fn.__name__ in available_fns:
                result = fn(*args, **kwargs)
            else:
                result = f'Function **{fn.__name__}** is not available for privilige level **{privilege_lvl}**'
            return result
        return inner
    return dec_privilege          
        

In [55]:
@dec_factory('high')
def add(user_input):
    name, coverage = user_input
    if name in user_list:
        response = f'{name} already exists, please use modify'
    else:
        user_list[name] = coverage
        response = f'{name} added with coverage : {coverage} in user list'
    return response

In [56]:
add(('Akshay','Vision'))

'Akshay added with coverage : Vision in user list'

In [57]:
add(('Akshay','Vision'))

'Akshay already exists, please use modify'

In [61]:
@dec_factory('low')
def add(user_input):
    name, coverage = user_input
    if name in user_list:
        response = f'{name} already exists, please use modify'
    else:
        user_list[name] = coverage
        response = f'{name} added with coverage : {coverage} in user list'
    return response

In [62]:
add(('Akshay','Vision'))

'Function add is not available for privilige level low'

In [63]:
@dec_factory('high')
def remove(user_input):
    name = user_input
    if name in user_list:
        user_list[name] = ()
        response = f'Coverage removed for {name}'
    else:
        response = f'{name} already not present in user list & hence no coverage exists'
    return response

In [64]:
user_list

{'Amit': 'All',
 'Amir': 'Medical',
 'Anil': 'Dental',
 'Sachin': 'Vision',
 'Akshay': 'Vision'}

In [65]:
remove('Akshay')

'Coverage removed for Akshay'

In [66]:
user_list

{'Amit': 'All',
 'Amir': 'Medical',
 'Anil': 'Dental',
 'Sachin': 'Vision',
 'Akshay': ()}

In [67]:
remove('Dravid')

'Dravid already not present in user list & hence no coverage exists'

In [68]:
@dec_factory('mid')
def remove(user_input):
    name = user_input
    if name in user_list:
        user_list[name] = ()
        response = f'Coverage removed for {name}'
    else:
        response = f'{name} already not present in user list & hence no coverage exists'
    return response

In [69]:
remove('Akshay')

'Function remove is not available for privilige level mid'

In [71]:
@dec_factory('mid')
def modify(user_input):
    name, new_coverage = user_input
    if name in user_list:
        old_coverage    = user_list[name]
        user_list[name] = new_coverage
        response = f'Coverage modified as new:{new_coverage} from old:{old_coverage} for {name}'
    else:
        response = f'{name} not present in user list. Please use **add**'
    return response

In [72]:
modify(('Akshay','Vision'))

'Coverage modified as new:Vision from old:() for Akshay'

In [73]:
modify(('Ajay','Dental'))

'Ajay not present in user list. Please use **add**'

In [74]:
@dec_factory('low')
def modify(user_input):
    name, new_coverage = user_input
    if name in user_list:
        old_coverage    = user_list[name]
        user_list[name] = new_coverage
        response = f'Coverage modified as new:{new_coverage} from old:{old_coverage} for {name}'
    else:
        response = f'{name} not present in user list. Please use **add**'
    return response

In [75]:
modify(('Akshay','Dental'))

'Function **modify** is not available for privilige level **low**'

In [78]:
@dec_factory('low')
def view(user_input):
    name = user_input
    if name in user_list:
        response = f'Name : {name}, Coverage : {user_list[name]} '
    else:
        response = f'{name} not present in user list.'
    return response

In [79]:
view('Akshay')

'Name : Akshay, Coverage : Vision '

In [80]:
view('Ajay')

'Ajay not present in user list.'

In [81]:
@dec_factory('no')
def view(user_input):
    name = user_input
    if name in user_list:
        response = f'Name : {name}, Coverage : {user_list[name]} '
    else:
        response = f'{name} not present in user list.'
    return response

In [82]:
view('Ajay')

'Function **view** is not available for privilige level **no**'


### Another Approach : 
***Start with a decorator_factory that takes an argument one of these strings, high, mid, low or no. Then write the decorator that has 4 free variables  based on the argument set by the factory call, give access to 4, 3, 2 or 1 arguments to the function being decorated from var1, var2, var3, var4***

In [105]:
def decorator_factory(privilege_lvl):

    def dec_privilege(fn):
        access = {'high':('add', 'remove', 'modify', 'view'), 'mid':('add', 'modify', 'view'), 'low':('view'),'no':()}
        def inner(*args, **kwargs):
            if privilege_lvl in access.keys():
                return access[privilege_lvl]
            else:
                return "Improper access keyword set"
        return inner
    return dec_privilege 

In [106]:
def dummy(*args):
    return args

In [107]:
decorator_factory('mid')(dummy)()

('add', 'modify', 'view')

In [108]:
len(decorator_factory('mid')(dummy)())

3

In [110]:
@decorator_factory('low')
def dummy2(*args):
    return args    

In [111]:
dummy2()

'view'

In [112]:
@decorator_factory('random')
def dummy3(*args):
    return args

dummy3()

'Improper access keyword set'

## Single Dispatch

In [1]:
from functools import singledispatch
from html import escape
@singledispatch
def htmlize(a):
    return escape(str(a))

In [2]:
@htmlize.register(int)
def html_int(a):
    return f'{a}(<i>{str(hex(a))}</i>)'

@htmlize.register(float)
def html_real(a):
    return f'{round(a, 2)}'

from decimal import Decimal
@htmlize.register(Decimal)
def html_real(a):
    return f'{round(a, 2)}'

def html_escape(arg):
    return escape(str(arg))

@htmlize.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_sequence(l):
    items = (f'<li>{html_escape(item)}</li>' for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = (f'<li>{k}={v}</li>' for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [3]:
htmlize('1 < 2')

'1 &lt; 2'

In [4]:
htmlize([1,2,3])

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n</ul>'

In [5]:
htmlize({1:'a', 2:'b'})

'<ul>\n<li>1=a</li>\n<li>2=b</li>\n</ul>'

In [6]:
htmlize((1,2,3))

'<ul>\n<li>1</li>\n<li>2</li>\n<li>3</li>\n</ul>'

In [7]:
htmlize(3.114789)

'3.11'

In [8]:
htmlize(2)

'2(<i>0x2</i>)'

In [9]:
htmlize(2.0)

'2.0'

### Working section

In [5]:
sec = datetime.now().second
type(sec)

int

In [ ]:
import timeit

In [6]:
def add10(a:int)-> int:
    '''
    Adds a number with 10
    '''
    return a + 10